In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random



In [6]:
intents = json.loads(open('intents.json', encoding='utf-8').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


In [12]:

def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    print(res)
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [13]:


import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 ))
    
        ints = predict_class(msg)
        print(ints)
        res = getResponse(ints, intents)
        
        ChatBox.insert(END, "Bot: " + res + '\n\n')
            
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)
 

root = Tk()
root.title("Aiolearn Chatbot")
root.geometry("400x500")
root.resizable(width=TRUE, height=TRUE)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="ارسال", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

root.mainloop()


1/1 [==============================] - 0s 16ms/step
[3.1747735e-05 2.4747902e-03 6.1956998e-05 6.7106004e-05 2.6980066e-04
 9.9542397e-01 1.0390331e-04 2.6783079e-04 5.5546738e-04 7.4337481e-04]
[{'intent': 'greeting', 'probability': '0.995424'}]
1/1 [==============================] - 0s 16ms/step
[5.7723528e-06 8.3312945e-05 4.4441767e-06 4.5516895e-06 1.4101129e-05
 9.9981934e-01 2.8540849e-06 3.2248576e-05 1.1383060e-05 2.1972519e-05]
[{'intent': 'greeting', 'probability': '0.99981934'}]
1/1 [==============================] - 0s 17ms/step
[5.6655704e-11 1.0000000e+00 1.0148130e-10 6.8352433e-12 8.6397989e-12
 7.2779460e-10 8.1344032e-10 5.9234624e-11 1.1157905e-09 3.5652110e-11]
[{'intent': 'aiolearn', 'probability': '1.0'}]
1/1 [==============================] - 0s 17ms/step
[1.22758115e-08 9.99996424e-01 2.52482621e-08 2.32614217e-09
 1.52630047e-10 2.34738504e-08 1.04883135e-07 5.00581450e-07
 2.99806538e-06 3.28483751e-09]
[{'intent': 'aiolearn', 'probability': '0.9999964'}]
1/1